In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.Resize(224),
                                transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='/content/data/',
                                             train=True,
                                             download=True,
                                             transform=transform)

test_dataset = torchvision.datasets.MNIST(root='/content/data/',
                                            train=False,
                                            transform=transform,
                                            download=True)

In [3]:
train_data = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=100,
                                         shuffle=True)

test_data = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=100,
                                        shuffle=False)

In [4]:
class AlexNet(nn.Module):
  
  def __init__(self):

    super(AlexNet,self).__init__()
    self.conv1 = nn.Conv2d(1,96,11,4,2)
    self.conv2 = nn.Conv2d(96,256,5,1,2)
    self.conv3 = nn.Conv2d(256,384,3,1,1)
    self.conv4 = nn.Conv2d(384,384,3,1,1)
    self.conv5 = nn.Conv2d(384,256,3,1,1)
    self.pool = nn.MaxPool2d(kernel_size=3,stride=2,padding=0)
    self.fc1 = nn.Linear(6*6*256,4096)
    self.fc2 = nn.Linear(4096,4096)
    self.output_layer = nn.Linear(4096,10)
    self.lbn = nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2)
    self.dropout = nn.Dropout(p=0.5,inplace=True)
    self.relu = nn.ReLU(inplace=False)
  
  def forward(self,x):

    # 1st layer
    x = self.conv1(x)
    x = self.relu(x)
    x = self.lbn(x)
    x = self.pool(x)
    

    # 2nd layer
    x = self.conv2(x)
    x = self.relu(x)
    x = self.lbn(x)
    x = self.pool(x)
    

    # 3rd Layer
    x = self.conv3(x)
    x = self.relu(x)

    # 4th Layer
    x = self.conv4(x)
    x = self.relu(x)

    # 5th layer
    x = self.conv5(x)
    x = self.relu(x)
    x = self.pool(x)

    # Flatten 
    x = x.view(x.size(0),-1)

    # 6th Layer
    x = self.dropout(x)
    x = self.fc1(x)
    x = self.relu(x)

    # 7th Layer
    x = self.dropout(x)
    x = self.fc2(x)
    x = self.relu(x)

    # 8th Layer (Output Layer)
    x = self.output_layer(x)

    return x

In [5]:
model = AlexNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
if torch.cuda.is_available():
  model = model.cuda()
summary(model,input_size=(1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          11,712
              ReLU-2           [-1, 96, 55, 55]               0
 LocalResponseNorm-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
 LocalResponseNorm-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,

In [6]:
epochs = 10
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(epochs):
  correct = 0
  iterations = 0
  iter = 0.0

  for batch , (images,labels) in enumerate(train_data):

    if torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()

    outputs = model(images)

    loss = loss_function(outputs,labels)
    iter = iter + loss.item()

    optimizer.zero_grad()

    loss.backward()
    optimizer.step()
    _,pred = torch.max(outputs,1)
    correct = correct + (labels==pred).sum().item()

    iterations +=1
  print('-----------EPOCH '+str(epoch+1)+'------------')
  train_loss.append(iter/iterations)
  train_acc.append(correct/len(train_dataset))
  print('TRAINING LOSS = ' + str(train_loss[epoch]))
  print('TRAINING ACCURACY = ' + str(train_acc[epoch]))

  correct = 0
  iter = 0.0
  iterations = 0

  for batch , (images,labels) in enumerate(test_data):
    if torch.cuda.is_available():
      images = images.cuda()
      labels = labels.cuda()

    outputs = model(images)

    _,pred = torch.max(outputs,1)
    correct = correct + (labels==pred).sum().item()
    loss = loss_function(outputs,labels)
    iter = iter + loss.item()
    iterations += 1

  test_loss.append(iter/iterations)
  test_acc.append(correct/len(test_dataset))
  print('TEST LOSS = ' + str(test_loss[epoch]))
  print('TESTING ACCURACY = ' + str(test_acc[epoch]))
  print('------------------------------------')

-----------EPOCH 1------------
TRAINING LOSS = 0.34788329200508694
TRAINING ACCURACY = 0.8815
TEST LOSS = 0.0803766034124419
TESTING ACCURACY = 0.974
------------------------------------
-----------EPOCH 2------------
TRAINING LOSS = 0.08363534129108302
TRAINING ACCURACY = 0.9753333333333334
TEST LOSS = 0.052078438482130875
TESTING ACCURACY = 0.9838
------------------------------------
-----------EPOCH 3------------
TRAINING LOSS = 0.06593589915292493
TRAINING ACCURACY = 0.9808833333333333
TEST LOSS = 0.04469609695195686
TESTING ACCURACY = 0.9852
------------------------------------
-----------EPOCH 4------------
TRAINING LOSS = 0.0564296290105752
TRAINING ACCURACY = 0.9829666666666667
TEST LOSS = 0.04725055487419013
TESTING ACCURACY = 0.9853
------------------------------------
-----------EPOCH 5------------
TRAINING LOSS = 0.05164653615385759
TRAINING ACCURACY = 0.98515
TEST LOSS = 0.04147081316317781
TESTING ACCURACY = 0.9872
------------------------------------
-----------EPOCH 6--